In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("OnlineRetail.csv", encoding='ISO-8859-1')
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"])
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
InvoiceNo      541909 non-null object
StockCode      541909 non-null object
Description    540455 non-null object
Quantity       541909 non-null int64
InvoiceDate    541909 non-null datetime64[ns]
UnitPrice      541909 non-null float64
CustomerID     406829 non-null float64
Country        541909 non-null object
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


## EDA

In [5]:
sorted(df.Quantity.unique())[:5]

[-80995, -74215, -9600, -9360, -9058]

"Quantity" values less than zero indicate that the products are returned. Hence we won't consider them in our analysis

In [6]:
df = df[df["Quantity"] > 0]

In [7]:
# Unique stock codes
df.StockCode.nunique()

3941

In [8]:
# Max and Min dates for all transactions
print(min(df.InvoiceDate))
max(df.InvoiceDate)

2010-12-01 08:26:00
2011-12-09 12:50:00


In [9]:
sorted(df.UnitPrice.unique())[0:5]

[-11062.06, 0.0, 0.001, 0.04, 0.06]

In [10]:
# Unit Price zero or less than than zero should be removed
df = df[df["UnitPrice"] > 0]

In [11]:
# Create new column which tells total cost per transaction

df["TotalPrice"] = df["UnitPrice"] * df["Quantity"]

In [12]:
"""
    To calculate Recency, we will need a reference date.
    Lets create a reference date which be the next day to last date of all the transactions mentioned above
    i.e. reference_date = Max_Date + 1
""" 

import datetime
reference_date = max(df.InvoiceDate) + datetime.timedelta(days=1)
reference_date

Timestamp('2011-12-10 12:50:00')

In [13]:
def get_recency(x):
    last_purchase = x.max()
    return (reference_date - last_purchase).days

In [14]:
# aggregate data by the customers
customers = df.groupby('CustomerID').agg(
    recency = ('InvoiceDate', get_recency),
    frequency = ('InvoiceNo', 'count'),
    monetary = ('TotalPrice', 'sum')
)

customers.head()

,recency,frequency,monetary
CustomerID,,,
12346.0,326,1,77183.60
12347.0,2,182,4310.00
12348.0,75,31,1797.24
12349.0,19,73,1757.55
12350.0,310,17,334.40


# Now for each customer, lets see in which quartile they fall into with respect to Recency, Frequency and Monetary

In [15]:
"""
    Recency quartile calculation
"""

r_labels = range(4, 0, -1) # Ranges from 4 to 1 (More less the recency value, higher their chances of being in earlier buckets)
recency = customers['recency']
r_quartiles, bins = pd.qcut(recency, 4, labels=r_labels, retbins=True)
customers = customers.assign(R=r_quartiles.values)

In [17]:
"""
    Frequency quartile calculation
"""

f_labels = range(1, 5)
frequency = customers['frequency'].rank(method='first') # rank to deal with duplicate values
f_quartiles, bins = pd.qcut(frequency, 4, labels=f_labels, retbins=True)
customers = customers.assign(F = f_quartiles.values)

In [18]:
"""
    Monetary quartile calculation
"""

m_labels = range(1, 5)
monetary = customers['monetary']
m_quartiles, bins = pd.qcut(monetary, 4, labels=m_labels, retbins=True)
customers = customers.assign(M = m_quartiles.values)

In [19]:
customers[['R', 'F', 'M']].head()

,R,F,M
CustomerID,,,
12346.0,1,1,4
12347.0,4,4,4
12348.0,2,2,4
12349.0,3,3,4
12350.0,1,1,2


# Here the RFM values ranges from 1-1-1(lowest) to 4-4-4(Highest).
# If we give equal priority to all the three i.e. R-F-M, then we can
# create a single metric called "score" which is addition of all metrics viz - RFM

In [22]:
customers['score'] = customers[['R','F','M']].sum(axis=1)
customers.head()

,recency,frequency,monetary,R,F,M,score
CustomerID,,,,,,,
12346.0,326,1,77183.60,1,1,4,6.0
12347.0,2,182,4310.00,4,4,4,12.0
12348.0,75,31,1797.24,2,2,4,8.0
12349.0,19,73,1757.55,3,3,4,10.0
12350.0,310,17,334.40,1,1,2,4.0


In [23]:
"""
    Now lets divide the customer base into three categories with scores
    ranging from 3 to 6 are Bronze, from 6 to 9 are Silver and from 9 to 12 are Gold.
"""

def get_tier(df):
    if df['score'] >= 9:
        return 'Gold'
    elif (df['score'] >= 6) and (df['score'] < 9):
        return 'Silver'
    else:
        return 'Bronze'

In [24]:
rfm = customers[["score"]]
rfm['tier'] = rfm.apply(get_tier, axis=1)
rfm.head()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,score,tier
CustomerID,,
12346.0,6.0,Silver
12347.0,12.0,Gold
12348.0,8.0,Silver
12349.0,10.0,Gold
12350.0,4.0,Bronze
